In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.corpus import cmudict
import os
import re

In [21]:
def sort_by_participant(debate):
    participant = {}
    current_speaker = ''
    for line in debate:
        line = line.strip()
        speaker = re.match("[A-Z]'?[A-Z]+:",line)
        if speaker:
            speaker = speaker.group(0).strip(':').strip()
            if 'MALLEY' in speaker:
                speaker = "O'MALLEY"
            if speaker not in participant.keys():
                participant[speaker] = []
            current_speaker = speaker
        else:
            if len(line) > 0:
                try:
                    participant[current_speaker].append(line)
                except:
                    print current_speaker
    return participant

def make_corpus(speech):
    joined_speech = []
    for turn in speech:
        line = turn
        line = re.sub('([a-zA-Z])[\./]([a-zA-Z])',r'\1 \2',line)
        line = re.sub('\[.*?\]','',line)
        # words = nltk.word_tokenize(line.strip())
        sent_tokenize_list = sent_tokenize(line)
        for sentence in sent_tokenize_list:
            sentence_words = []
            words = word_tokenize(sentence.strip())
            for word in words:
                word = word.lower().strip().strip('.').strip(':')
                if len(word) != 0:
                    if (word != "``") and (word != "--") and (word != "'"):
                        if re.search('.*[^a-zA-Z0-9\'\-]',word):
                            if (len(word) >= 2):
                                sentence_words.append(word)
                                # joined_speech.append(word)
                        else:
                            sentence_words.append(word)
                            # joined_speech.append(word)
            joined_speech.append(sentence_words)
    return joined_speech

def get_corpus(participant):
    corpus = {}
    for person in participant.keys():
        corpus[person] = make_corpus(participant[person])
    return corpus

In [22]:
for f in os.listdir('debates'):
    if re.search('[RD].*201[56]',f):
    #if re.search('[RD].*200[78]',f):
        raw = []
        ifile = open('debates/'+f,'r')
        raw = ifile.readlines()
        ifile.close()

In [23]:
debate = []
for line in raw:
    line = line.decode('utf8', 'ignore')
    line = line.encode('ascii', 'ignore')
    debate.append(line)

In [24]:
participant = sort_by_participant(debate)
corpus = get_corpus(participant)

In [28]:
text = corpus['TRUMP']

In [ ]:
f = open('dearsister_cut.txt')
raw = f.read()
raw = raw.decode('utf8', 'ignore')
raw = raw.encode('ascii', 'ignore')
sent_tokenize_list = sent_tokenize(raw)
text = []
for sentence in sent_tokenize_list:
    words = word_tokenize(sentence)
    text.append(words)

In [ ]:
import nltk

# Functions to generate LMs:
def get_unigram(sents):
    fd = nltk.FreqDist()
    for sent in sents:
        for word in sent:
            fd[word] += 1    

    return nltk.MLEProbDist(fd)

def get_bigram(sents):
    cfd = nltk.ConditionalFreqDist()
    for sent in sents:
        prev = '<s>'
        for word in sent:
            cfd[prev][word] += 1
            prev = word
        cfd[word]['</s>'] += 1
    return nltk.ConditionalProbDist(cfd, nltk.MLEProbDist)

def get_trigram(sents):
    cfd = nltk.ConditionalFreqDist()
    for sent in sents:
        cond1 = '<s>'
        cond2 = '<s>'
        for word in sent:
            context = " ".join([cond1, cond2])
            cfd[context][word] += 1
            cond1 = cond2
            cond2 = word
    return nltk.ConditionalProbDist(cfd, nltk.MLEProbDist)


# Functions to generate random sentences based on LM:
def generate_random_unigram(n, model):
    words = []
    for i in range(n):
        word = model.generate()
        words.append(word)
    return " ".join(words)
    
def generate_random_bigram(n, model):
    words = []
    condition = '<s>'
    for i in range(n):
        word = model[condition].generate()
        if word == '</s>':
            condition = '<s>'
        else:
            words.append(word)
            condition = word
    return " ".join(words)

def generate_random_trigram(n, model):
    words = []
    condition = '<s> <s>'
    for i in range(n):
        try:
            word = model[condition].generate()
            if word in ['.', '?', '!']:
                words.append(word)
                condition = '<s> <s>'
            else:
                words.append(word)
                condition = " ".join([condition.split()[-1], word])
        except:
            pass
    return " ".join(words)

In [32]:
import sys
import random

def text_loop(text_dict):
    
    text_no = len(text_dict.keys()) - 1
    text_key = None
    print "Choose a text from the following list: "
    for key, value in text_dict.items():
        print "   %s: %s" % (key, value)

    print ""
    while text_key == None:

        text_key = int(raw_input("Please enter a number between 0-%s: " % text_no))

        if text_key > max(text_dict.keys()):
            text_key = None
            print "Not a valid option.",

    return text_key

def prompt_lm(text):
    lm = raw_input("Choose a LM. Enter 'unigram', 'bigram' or 'trigram': ")
    if lm == 'unigram':
        model = get_unigram(text)
        generator = generate_random_unigram
        return model, generator
    elif lm == 'bigram':
        model = get_bigram(text)
        generator = generate_random_bigram
        return model, generator
    elif lm == 'trigram':
        model = get_trigram(text)
        generator = generate_random_trigram
        return model, generator
    else:
        sys.exit("Something is wrong. Try again.")

def sentence_generation(model, generator):
    length = None
    while not length == 0:
        length = int(raw_input(
            "Enter the length of sentence you'd like to generate (or 0 to exit): "))
        print '\n' + generator(length, model) + '\n'
        
def make_haiku(model, generator):
    
    nsyls = 0
    
    while nsyls != 5:
        length = random.randint(1,5)
        line1 = generator(length, model)
        nsyls = nsyl(line1)
    
    nsyls = 0
    while nsyls != 7:
        nsyls = 0
        length = random.randint(1,7)
        line2 = generator(length, model)
        nsyls = nsyl(line2)
    
    nsyls = 0
    while nsyls != 5:
        length = random.randint(1,5)
        line3 = generator(length, model)
        nsyls = nsyl(line3)
        
    print "\n{0}\n{1}\n{2}".format(line1, line2, line3)

def nsyl(sentence):
    d = cmudict.dict()
    words = sentence.split(' ')
    count = 0
    for word in words:
        try:
            count += [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
        except:
            count = 0
    return count

def main(text):
    model, generator = prompt_lm(text)
    #sentence_generation(model, generator)
    make_haiku(model, generator)

    prompt = raw_input("Do you want to try again (type 'yes' or 'no'): ")
    if prompt == 'yes':
        main(text)
    else:
        print "Goodbye!"

In [41]:
len(text)

354

In [ ]:
main(text)

Choose a LM. Enter 'unigram', 'bigram' or 'trigram': trigram
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(2, 0)
(2, 1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(3, 0)
(3, 1)
(3, 2)
(1, 0)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(2, 0)
(2, 1)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(3, 0)
(3, 1)
(3, 2)
(2, 0)
(2, 1)
(3, 0)
(3, 1)
(3, 2)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(4, 0)
(4, 1)
(4, 2)
(4, 3)

i say not in a
i 'm a businessman did
got along with all
Do you want to try again (type 'yes' or 'no'): yes
